In [1]:
# !python3 -m pip uninstall tensorflow tensorflow-probability nsc -y
# !python3 -m pip install tensorflow tensorflow-probability -q

In [2]:
# !python3 -m pip uninstall nsc -y -q
# !python3 -m pip install -i https://test.pypi.org/simple/ nsc -q

In [3]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import scipy
from scipy.special import beta, gamma
from collections import defaultdict
from typing import List

# From nsc lib
import nsc
from nsc import distributions as nsd
from nsc.util import function as nsc_func
from nsc.util.function import coupled_logarithm, coupled_exponential
# nsd = nsc.distributions

Importing NSC lib v0.0.2.16.


### 1) CoupledNormalDistribution function version

In [4]:
def norm_CG(sigma, kappa):
    if kappa == 0:
        result = math.sqrt(2*math.pi) * sigma
    elif kappa < 0:
        result = math.sqrt(math.pi) * sigma * math.gamma((-1+kappa) / (2*kappa)) / float(math.sqrt(-1*kappa) * math.gamma(1 - (1 / (2*kappa))))
    else:
        result = math.sqrt(math.pi) * sigma * math.gamma(1 / (2*kappa)) / float(math.sqrt(kappa) * math.gamma((1+kappa)/(2*kappa)))
  
    return result

In [5]:
def CoupledNormalDistribution(mean, sigma, kappa, alpha):
    """
    Short description
    
    Inputs
    ----------
    x : Input variable in which the coupled logarithm is applied to.
    mean : 
    sigma : 
    kappa : Coupling parameter which modifies the coupled logarithm function.
    dim : The dimension of x, or rank if x is a tensor. Not needed?
    """

    assert sigma > 0, "std must be greater than 0."
    assert alpha in [1, 2], "alpha must be set to either 1 or 2."

    
    if kappa >= 0:
        input = np.arange(mean-20, mean+20, (20+mean - -20+mean)/(2**16+1))
    else:
        x1 = mean - ((-1*sigma**2) / kappa)**0.5
        x2 = mean + ((-1*sigma**2) / kappa)**0.5
        input = np.arange(mean - ((-1*sigma**2) / kappa)**0.5, mean + ((-1*sigma**2) / kappa)**0.5, (x2-x1)/(2**16+1))
 
    normCGvalue = 1/float(norm_CG(sigma, kappa))
    
    coupledNormalDistributionResult = normCGvalue * (coupled_exponential((input - mean)**2/sigma**2, kappa)) ** -0.5
  
    return coupledNormalDistributionResult

In [6]:
kappa, alpha, dim = 0.5, 2, 1

In [7]:
mu, sigma = 0, 1 # mean and standard deviation
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
y = CoupledNormalDistribution(mu, sigma, kappa, alpha)

dx = np.arange(mu-20, mu+20, (20+mu - -20+mu)/(2**16+1))[1] - np.arange(mu-20, mu+20, (20+mu - -20+mu)/(2**16+1))[0]

### 2) CoupledNormalDistribution class

In [8]:
import numpy as np
import math
from typing import List


# From nsc lib
import nsc
from nsc import distributions as nsd
from nsc.util import function as nsc_func
from nsc.util.function import coupled_logarithm, coupled_exponential
# nsd = nsc.distributions

import ipdb

In [9]:
class CoupledNormal:
    """Coupled Normal Distribution.

    This distribution has parameters: location `loc`, 'scale', coupling `kappa`,
    and `alpha`.

    """
    def __init__(self,
                 loc: [int, float, List, np.ndarray],
                 scale: [int, float, List, np.ndarray],
                 kappa: [int, float] = 0.,
                 alpha: int = 2,
                 validate_args: bool = True
                 ):
        loc = np.asarray(loc) if isinstance(loc, List) else loc
        scale = np.asarray(scale) if isinstance(scale, List) else scale
        if validate_args:
            assert isinstance(loc, (int, float, np.ndarray)), "loc must be either an int/float type for scalar, or an list/ndarray type for multidimensional."
            assert isinstance(scale, (int, float, np.ndarray)), "scale must be either an int/float type for scalar, or an list/ndarray type for multidimensional."
            assert type(loc) == type(scale), "loc and scale must be the same type."
            if isinstance(loc, np.ndarray):
                # assert loc.shape == scale.shape, "loc and scale must have the same dimensions (check respective .shape())."
                assert np.all((scale >= 0)), "All scale values must be greater or equal to 0."            
            else:
                assert scale >= 0, "scale must be greater or equal to 0."            
            assert isinstance(kappa, (int, float)), "kappa must be an int or float type."
            assert isinstance(alpha, int), "alpha must be an int that equals to either 1 or 2."
            assert alpha in [1, 2], "alpha must be equal to either 1 or 2."
        self._loc = loc
        self._scale = scale
        self._kappa = kappa
        self._alpha = alpha
        self._dim = self._n_dim()

    @property
    def loc(self):
        return self._loc

    @property
    def scale(self):
        return self._scale

    @property
    def kappa(self):
        return self._kappa

    @property
    def alpha(self):
        return self._alpha

    @property
    def dim(self):
        return self._dim
    
    def _n_dim(self):
        return 1 if self._event_shape() == [] else self._event_shape()[0]

    def _batch_shape(self) -> List:
        if self._rank(self._loc) == 0:
            # return [] signifying single batch of a single distribution
            return []
        else:
            # return the batch shape in list format
            return list(self._loc.shape)

    def _event_shape(self) -> List:
        # For univariate Coupled Normal distribution, event shape is always []
        # [] signifies single random variable dim (regardless of batch size)
        return []

    def _rank(self, value: [int, float, np.ndarray]) -> int:
        # specify the rank of a given value, with rank=0 for a scalar and rank=ndim for an ndarray
        if isinstance(value, (int, float)):
            return 0 
        else:
            return len(value.shape)

    def sample_n(self, n: int) -> np.array:
        # https://het.as.utexas.edu/HET/Software/Numpy/reference/generated/numpy.random.gamma.html
        normal_samples = np.random.normal(loc=self._loc, scale=self._scale, size=n)
        if self._kappa == 0:
            samples = normal_samples
        else:
            gamma_samples = np.random.gamma(shape=1/(2*self._kappa), scale=self._scale, size=n)
            samples = normal_samples * 1/np.sqrt(gamma_samples*self._kappa)
        return samples * self._scale + self._loc
        '''
        normal_seed, gamma_seed = samplers.split_seed(seed, salt='student_t')
        shape = ps.concat([[n], batch_shape], 0)

        normal_sample = samplers.normal(shape, dtype=dtype, seed=normal_seed)
        df = df * tf.ones(batch_shape, dtype=dtype)
        gamma_sample = gamma_lib.random_gamma(
            [n], concentration=0.5 * df, rate=0.5, seed=gamma_seed)
        samples = normal_sample * tf.math.rsqrt(gamma_sample / df)
        return samples * scale + loc
        '''

    def prob(self, X: [List, np.ndarray]) -> np.ndarray:
        # Check whether input X is valid
        X = np.asarray(X) if isinstance(X, List) else X
        assert isinstance(X, np.ndarray), "X must be a List or np.ndarray."
        # assert type(X[0]) == type(self._loc), "X samples must be the same type as loc and scale."
        if isinstance(X[0], np.ndarray):
            assert X[0].shape == self._loc.shape, "X samples must have the same dimensions as loc and scale (check respective .shape())."
        # Calculate PDF with input X
        X_norm = (X-self._loc)**2 / self._scale**2
        norm_term = self._normalized_term()
        p = (coupled_exponential(X_norm, kappa=self._kappa))**-0.5 / norm_term
        # normCGvalue =  1/float(norm_CG(scale, kappa))
        # coupledNormalDistributionResult = normCGvalue * (coupled_exponential(y, kappa)) ** -0.5
        return p

    '''
    def _normalized_term(self) -> [int, float, np.ndarray]:
        norm_term = np.sqrt(2*np.pi) * self.scale
        return self._normalization_function(self.scale)
    '''

    # Normalization constant of 1-D Coupled Gaussian (NormCG)
    def _normalized_term(self, use_beta_func: bool = False, use_scipy_gamma_func: bool = False) -> [int, float, np.ndarray]:
        if use_beta_func:
            base_term = np.sqrt(2*np.pi) * self._scale
            return base_term*self._normalization_function()
        elif use_scipy_gamma_func:
            if self._kappa == 0:
                norm_term = np.sqrt(2*np.pi) * self._scale
            elif self._kappa < 0:
                gamma_num = gamma(self._kappa-1) / (2*self._kappa)
                gamma_dem = gamma(1 - (1 / (2*self._kappa)))
                norm_term = (np.sqrt(np.pi)*self.scale*gamma_num) / float(np.sqrt(-1*self.kappa)*gamma_dem)
            else:
                gamma_num = gamma(1 / (2*self.kappa))
                gamma_dem = gamma((1+self.kappa)/(2*self._kappa))
                norm_term = (np.sqrt(np.pi)*self.scale*gamma_num) / float(np.sqrt(self.kappa)*gamma_dem)
        else:
            if self._kappa == 0:
                norm_term = np.sqrt(2*np.pi) * self._scale
            elif self._kappa < 0:
                gamma_num = math.gamma(self.kappa-1) / (2*self._kappa)
                gamma_dem = math.gamma(1 - (1 / (2*self._kappa)))
                norm_term = (np.sqrt(np.pi)*self.scale*gamma_num) / float(np.sqrt(-1*self.kappa)*gamma_dem)
            else:
                gamma_num = math.gamma(1 / (2*self._kappa))
                gamma_dem = math.gamma((1+self._kappa)/(2*self._kappa))
                norm_term = (np.sqrt(np.pi)*self.scale*gamma_num) / float(np.sqrt(self.kappa)*gamma_dem)
        return norm_term

    def _normalization_function(self):
        k, d = self._kappa, self._dim
        assert -1/d < k, "kappa must be greater than -1/dim."
        if k == 0:
            return 1
        elif k > 0:
            func_term = (1 + d*k) / (2*k)**(d/2)
            beta_input_x = 1/(2*k) + 1
            beta_input_y = d/2
            gamma_input = d/2
            return func_term * beta(beta_input_x, beta_input_y)/gamma(gamma_input)
        else:  # -1 < self.kappa < 0:
            func_term = 1 / (-2*k)**(d/2)
            beta_input_x = (1 + d*k)/(-2*k) + 1
            beta_input_y = d/2
            gamma_input = d/2
            return func_term * beta(beta_input_x, beta_input_y)/gamma(gamma_input)
    
    def __repr__(self) -> str:
        return f"<nsc.distributions.{self.__class__.__name__} batch_shape={str(self._batch_shape())} event_shape={str(self._event_shape())}>"


In [10]:
class MultivariateCoupledNormal(CoupledNormal):
    """Multivariate Coupled Normal Distribution.

    This distribution has parameters: location `loc`, 'scale', coupling `kappa`,
    and `alpha`.

    """
    def __init__(self,
                 loc: [int, float, List, np.ndarray],
                 scale: [int, float, List, np.ndarray],
                 kappa: [int, float] = 0.,
                 alpha: int = 2,
                 validate_args: bool = True
                 ):
        if validate_args:
            assert isinstance(loc, (list, np.ndarray)), "loc must be either a list or ndarray type. Otherwise use CoupledNormal."
            assert isinstance(scale, (list, np.ndarray)), "scale must be either a list or ndarray type. Otherwise use CoupledNormal."
        super(MultivariateCoupledNormal, self).__init__(
            loc=loc,
            scale=scale,
            kappa=kappa,
            alpha=alpha,
            validate_args=validate_args
        )
        if self._rank(self._scale) == 1:
            self._scale = np.diag(self._scale)
        # Ensure that scale is indeed positive definite
        assert self.is_positive_definite(self._scale), "scale must be positive definite, but not necessarily symmetric."

    # Credit: https://stackoverflow.com/questions/16266720/find-out-if-matrix-is-positive-definite-with-numpy
    # This is only for positive definite, not symmetric positive definite
    def is_positive_definite(self, A: np.ndarray) -> bool:
        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False

    def _batch_shape(self) -> List:
        if self._rank(self._loc) == 1:
            # return [] signifying single batch of a single distribution
            return []
        else:
            # return [batch size]
            return list(self._loc.shape[:-1])

    def _event_shape(self) -> List:
        if self._rank(self._loc) == 1:
            # if loc is only a vector
            return list(self._loc.shape)
        else:
            # return [n of random variables] when rank >= 2
            return [self._loc.shape[-1]]

    def _rank(self, value: [int, float, np.ndarray]) -> int:
        # specify the rank of a given value, with rank=0 for a scalar and rank=ndim for an ndarray
        if isinstance(value, (int, float)):
            return 0 
        else:
            return len(value.shape)

    def sample_n(self, n: int) -> np.array:
        pass
        '''
        normal_seed, chi2_seed = samplers.split_seed(seed, salt='multivariate t')

        loc = tf.broadcast_to(self._loc, self._sample_shape())
        mvn = mvn_linear_operator.MultivariateNormalLinearOperator(
            loc=tf.zeros_like(loc), scale=self._scale)
        normal_samp = mvn.sample(n, seed=normal_seed)

        df = tf.broadcast_to(self.df, self.batch_shape_tensor())
        chi2 = chi2_lib.Chi2(df=df)
        chi2_samp = chi2.sample(n, seed=chi2_seed)

        return (self._loc +
                normal_samp * tf.math.rsqrt(chi2_samp / self._df)[..., tf.newaxis])
        '''

    def prob(self, X: [List, np.ndarray], beta_func: bool = True) -> np.ndarray:
        assert X.shape[-1] ==  self._loc.shape[-1], "input X and loc must have the same dims."
        sigma = np.matmul(self._scale, self._scale)
        sigma_inv = np.linalg.inv(sigma)
        _normalized_X = lambda x: np.linalg.multi_dot([x, sigma_inv, x])
        X_norm = np.apply_along_axis(_normalized_X, 1, X)
        norm_term = self._normalized_term(beta_func)
        p = (coupled_exponential(X_norm, kappa=self._kappa, dim=self._dim))**(-1/self._alpha) / norm_term
        return p

    # Normalization constant of the multivariate Coupled Gaussian (NormMultiCoupled)
    def _normalized_term(self, beta_func: bool = False, scipy_gamma_func: bool = False) -> [int, float, np.ndarray]:
        if beta_func:
            sigma = np.matmul(self._scale, self._scale.T)
#             sigma = np.matmul(self._scale, self._scale)
            sigma_det = np.linalg.det(sigma)
#             k, d = self._kappa, self._dim
#             assert -1/d < k, "kappa must be greater than -1/dim."
#             base_term = np.sqrt(2 * np.pi * sigma_det)
#             base_term = (2*np.pi)**(self.dim/2) * (sigma_det)**(1/2)
            base_term = np.sqrt((2 * np.pi)**self._dim * sigma_det)
            return base_term*self._normalization_function()
            '''
            if k == 0:
                base_term = (2*np.pi)**(d/2) * (sigma_det)**(1/2)
                return base_term
            elif k > 0:
                base_term = (np.pi)**(d/2) * (sigma_det)**(1/2)
                func_term = (1 + d*k) / (k)**(d/2)
                beta_input_x = 1/(2*k) + 1
                beta_input_y = d/2
                gamma_input = d/2
                return base_term * func_term * beta(beta_input_x, beta_input_y)/gamma(gamma_input)
            else:  # -1 < self.kappa < 0:
                base_term = (np.pi)**(d/2) * (sigma_det)**(1/2)
                func_term = 1 / (-k)**(-d/2)
                beta_input_x = (1 + d*k)/(-2*k) + 1
                beta_input_y = d/2
                gamma_input = d/2
                return base_term * func_term * beta(beta_input_x, beta_input_y)/gamma(gamma_input)
            '''
        elif scipy_gamma_func:
            sigma = np.matmul(self._scale, self._scale.T)
            sigma_det = np.linalg.det(sigma)
            if self.alpha == 1:
                return sigma_det**0.5 / (1 + (-1 + self._dim)*self._kappa)
            else:  # self.alpha == 2
                gamma_num = gamma((1 + (-1 + self.dim)*self._kappa) / (2*self._kappa))
                gamma_dem = gamma((1 + self.dim*self._kappa) / (2*self._kappa))
                return (np.sqrt(np.pi) * sigma_det**0.5 * gamma_num) / (np.sqrt(self._kappa) * gamma_dem)
        else:
            sigma = np.matmul(self._scale, self._scale.T)
            sigma_det = np.linalg.det(sigma)
            if self.alpha == 1:
                return sigma_det**0.5 / (1 + (-1 + self._dim)*self._kappa)
            else:  # self.alpha == 2
                gamma_num = math.gamma((1 + (-1 + self._dim)*self._kappa) / (2*self._kappa))
                gamma_dem = math.gamma((1 + self._dim*self._kappa) / (2*self._kappa))
                return (np.sqrt(np.pi) * sigma_det**0.5 * gamma_num) / (np.sqrt(self._kappa) * gamma_dem)


***Test***

In [11]:
loc, scale, kappa, alpha = 0., 1., 0.5, 2
# X_input = np.arange(mu-20., mu+20., (20.+mu - -20.+mu)/(2.**16.+1.), dtype=float)
six_scale = 6.*scale
# X_input = np.arange(mu-6.*sigma, mu+6.*sigma, (20.+mu - -20.+mu)/(2.**16.+1.), dtype=float)
X_input = np.linspace(mu-six_scale, mu+six_scale, 1000)

In [12]:
X_input.shape

(1000,)

Coupled Normal distribution

In [13]:
cn = CoupledNormal(loc=mu, scale=sigma, kappa=kappa)

In [14]:
cn

<nsc.distributions.CoupledNormal batch_shape=[] event_shape=[]>

In [15]:
print(cn.dim)
print(cn._normalized_term())

1
2.82842712474619


In [16]:
cn.sample_n(1000)

array([ 1.73148159e+00, -3.02842407e+00, -3.02475038e-01,  8.53571749e-02,
       -1.06455611e+01,  1.98459437e-01,  3.47157819e-01,  4.28947512e-01,
        1.28000285e+00,  7.62771828e+00, -5.50429059e-01,  1.65250816e+00,
       -5.32556027e-02, -7.81403658e-03, -3.36104189e-01,  2.46400349e-01,
        5.02369439e+00,  2.59846729e-01,  5.63802310e-01,  6.39521852e-01,
       -3.03033687e+00, -1.80851664e+00,  4.70805311e+00,  9.52742517e-01,
       -7.05882901e-01,  1.14030580e+00, -2.71562691e+00,  4.02768211e+00,
       -1.34319755e+00,  1.39914854e+00, -8.59134845e-01,  7.72627626e-01,
        4.71821617e-01,  5.84516285e-01,  5.89152277e-01,  5.94538259e-02,
        5.43955443e+00,  2.87499899e-01, -9.59534784e-01,  4.73359125e+00,
       -1.10386865e+00,  8.26605370e-01, -3.17546701e+00,  1.56555667e+00,
       -8.41676405e-01,  3.43119241e+00,  3.21049709e+00,  3.12976253e-01,
       -1.54976068e+00,  3.67610632e+00,  1.83940227e+01, -1.12293090e+00,
       -4.47741569e-01, -

In [17]:
cn.prob(X_input)

array([0.00426898, 0.00429337, 0.00431793, 0.00434268, 0.00436762,
       0.00439274, 0.00441806, 0.00444357, 0.00446927, 0.00449516,
       0.00452126, 0.00454755, 0.00457405, 0.00460075, 0.00462765,
       0.00465476, 0.00468208, 0.00470961, 0.00473736, 0.00476532,
       0.00479349, 0.00482189, 0.0048505 , 0.00487934, 0.00490841,
       0.0049377 , 0.00496723, 0.00499698, 0.00502697, 0.0050572 ,
       0.00508766, 0.00511837, 0.00514932, 0.00518052, 0.00521196,
       0.00524366, 0.00527561, 0.00530781, 0.00534027, 0.005373  ,
       0.00540599, 0.00543924, 0.00547276, 0.00550656, 0.00554063,
       0.00557497, 0.0056096 , 0.0056445 , 0.0056797 , 0.00571518,
       0.00575095, 0.00578701, 0.00582338, 0.00586004, 0.005897  ,
       0.00593428, 0.00597186, 0.00600975, 0.00604796, 0.00608649,
       0.00612534, 0.00616451, 0.00620401, 0.00624385, 0.00628402,
       0.00632453, 0.00636538, 0.00640658, 0.00644813, 0.00649003,
       0.00653229, 0.00657491, 0.0066179 , 0.00666125, 0.00670

In [18]:
cn._normalized_term(), cn._normalized_term(use_scipy_gamma_func=True), cn._normalized_term(use_beta_func=True)

(2.82842712474619, 2.82842712474619, 2.82842712474619)

Test for really low kappa value

In [19]:
kappa=0.003

In [20]:
cn = CoupledNormal(loc=mu, scale=sigma, kappa=kappa)

In [21]:
print(1 / (2*kappa))
math.gamma(100)

166.66666666666666


9.332621544394415e+155

In [22]:
cn._normalized_term()

2.508508948180413

In [23]:
cn._normalized_term(use_beta_func=True)

2.5085089481804146

In [24]:
cn._normalized_term(use_beta_func=True)

2.5085089481804146

In [25]:
print(cn.dim)
print(cn._normalized_term())
print(cn.prob(X_input))

1
2.508508948180413
[1.42896327e-08 1.52522011e-08 1.62778955e-08 1.73707354e-08
 1.85349883e-08 1.97751838e-08 2.10961296e-08 2.25029279e-08
 2.40009925e-08 2.55960675e-08 2.72942464e-08 2.91019925e-08
 3.10261610e-08 3.30740213e-08 3.52532813e-08 3.75721129e-08
 4.00391789e-08 4.26636611e-08 4.54552902e-08 4.84243777e-08
 5.15818489e-08 5.49392780e-08 5.85089250e-08 6.23037750e-08
 6.63375792e-08 7.06248983e-08 7.51811481e-08 8.00226476e-08
 8.51666701e-08 9.06314963e-08 9.64364708e-08 1.02602061e-07
 1.09149921e-07 1.16102954e-07 1.23485386e-07 1.31322836e-07
 1.39642391e-07 1.48472692e-07 1.57844013e-07 1.67788354e-07
 1.78339533e-07 1.89533287e-07 2.01407373e-07 2.14001678e-07
 2.27358336e-07 2.41521846e-07 2.56539199e-07 2.72460013e-07
 2.89336668e-07 3.07224461e-07 3.26181751e-07 3.46270130e-07
 3.67554585e-07 3.90103682e-07 4.13989749e-07 4.39289077e-07
 4.66082123e-07 4.94453727e-07 5.24493340e-07 5.56295261e-07
 5.89958884e-07 6.25588967e-07 6.63295895e-07 7.03195978e-07
 7.4

Coupled Normal multiple distributions with batch vector size of 2

In [26]:
cn = CoupledNormal(loc=[0., 1.], scale=[1., 2.])
cn

<nsc.distributions.CoupledNormal batch_shape=[2] event_shape=[]>

In [27]:
print(cn.dim)
print(cn._normalized_term())
# print(cn.prob(X_input))

1
[2.50662827 5.01325655]


Coupled Normal multiple distributions with batch matrix size of 3x2

In [28]:
cn = CoupledNormal(loc=[[0., 1.], [0., 1.], [0., 1.]], scale=[[1., 2.], [1., 2.], [1., 2.]])
cn

<nsc.distributions.CoupledNormal batch_shape=[3, 2] event_shape=[]>

**Multivariate Coupled Normal distribution when d=2**

In [53]:
# batch_shape is [] while event_shape is [2]
mcn = MultivariateCoupledNormal(loc=[0., 1.], scale=[1., 2.], kappa=0.1)
mcn

<nsc.distributions.MultivariateCoupledNormal batch_shape=[] event_shape=[2]>

In [54]:
print(mcn.dim)
print(mcn.loc)
print(mcn.scale)
print(mcn.kappa)
print(mcn.alpha)

2
[0. 1.]
[[1. 0.]
 [0. 2.]]
0.1
2


In [55]:
mcn._normalized_term()

4.889680162071525

Create 2nd X variable from -12 to 12.

In [56]:
twelve_scale = 12.*scale
# X_input = np.arange(mu-6.*sigma, mu+6.*sigma, (20.+mu - -20.+mu)/(2.**16.+1.), dtype=float)
X2_input = np.linspace(mu-twelve_scale, mu+twelve_scale, 1000)

In [57]:
X_2D = np.column_stack((X_input, X2_input))
X_2D.shape

(1000, 2)

In [58]:
X_2D

array([[ -6.        , -12.        ],
       [ -5.98798799, -11.97597598],
       [ -5.97597598, -11.95195195],
       ...,
       [  5.97597598,  11.95195195],
       [  5.98798799,  11.97597598],
       [  6.        ,  12.        ]])

In [59]:
mcn._normalized_term(), mcn._normalized_term(beta_func=True)

(4.889680162071525, 12.566370614359169)

Here is the dataset of X_2D, we can see that the sample size (m) is 1000 and the dim (d) for each sample is 2.

In [60]:
mcn.loc, mcn.scale

(array([0., 1.]),
 array([[1., 0.],
        [0., 2.]]))

In [63]:
X_2D.shape[-1], mcn.loc.shape[-1]

(2, 2)

In [64]:
mcn.prob(X_2D)

(array([2.61762254e-07, 2.67346858e-07, 2.73059346e-07, 2.78902843e-07,
        2.84880553e-07, 2.90995764e-07, 2.97251847e-07, 3.03652264e-07,
        3.10200565e-07, 3.16900394e-07, 3.23755490e-07, 3.30769687e-07,
        3.37946924e-07, 3.45291240e-07, 3.52806782e-07, 3.60497804e-07,
        3.68368674e-07, 3.76423874e-07, 3.84668006e-07, 3.93105791e-07,
        4.01742078e-07, 4.10581843e-07, 4.19630193e-07, 4.28892375e-07,
        4.38373771e-07, 4.48079909e-07, 4.58016464e-07, 4.68189265e-07,
        4.78604293e-07, 4.89267692e-07, 5.00185772e-07, 5.11365010e-07,
        5.22812059e-07, 5.34533753e-07, 5.46537107e-07, 5.58829328e-07,
        5.71417820e-07, 5.84310183e-07, 5.97514228e-07, 6.11037976e-07,
        6.24889667e-07, 6.39077767e-07, 6.53610971e-07, 6.68498215e-07,
        6.83748675e-07, 6.99371784e-07, 7.15377231e-07, 7.31774973e-07,
        7.48575239e-07, 7.65788542e-07, 7.83425686e-07, 8.01497773e-07,
        8.20016211e-07, 8.38992728e-07, 8.58439375e-07, 8.783685

In [65]:
 mcn.prob(X_2D, beta_func=True)

array([2.61762254e-07, 2.67346858e-07, 2.73059346e-07, 2.78902843e-07,
       2.84880553e-07, 2.90995764e-07, 2.97251847e-07, 3.03652264e-07,
       3.10200565e-07, 3.16900394e-07, 3.23755490e-07, 3.30769687e-07,
       3.37946924e-07, 3.45291240e-07, 3.52806782e-07, 3.60497804e-07,
       3.68368674e-07, 3.76423874e-07, 3.84668006e-07, 3.93105791e-07,
       4.01742078e-07, 4.10581843e-07, 4.19630193e-07, 4.28892375e-07,
       4.38373771e-07, 4.48079909e-07, 4.58016464e-07, 4.68189265e-07,
       4.78604293e-07, 4.89267692e-07, 5.00185772e-07, 5.11365010e-07,
       5.22812059e-07, 5.34533753e-07, 5.46537107e-07, 5.58829328e-07,
       5.71417820e-07, 5.84310183e-07, 5.97514228e-07, 6.11037976e-07,
       6.24889667e-07, 6.39077767e-07, 6.53610971e-07, 6.68498215e-07,
       6.83748675e-07, 6.99371784e-07, 7.15377231e-07, 7.31774973e-07,
       7.48575239e-07, 7.65788542e-07, 7.83425686e-07, 8.01497773e-07,
       8.20016211e-07, 8.38992728e-07, 8.58439375e-07, 8.78368539e-07,
      

***Multivariate Coupled Normal distribution when d=8***

In [37]:
# batch_shape is [] while event_shape is [2]
mcn = MultivariateCoupledNormal(loc=[0., 1., 2., 3., 4., 5., 6., 7.],
                                scale=[1., 2., 3., 4., 5., 6., 7., 8.],
                                kappa=0.1
                               )
mcn

<nsc.distributions.MultivariateCoupledNormal batch_shape=[] event_shape=[8]>

In [38]:
print(mcn.dim)
print(mcn.loc)
print(mcn.scale)
print(mcn.kappa)
print(mcn.alpha)

8
[0. 1. 2. 3. 4. 5. 6. 7.]
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 3. 0. 0. 0. 0. 0.]
 [0. 0. 0. 4. 0. 0. 0. 0.]
 [0. 0. 0. 0. 5. 0. 0. 0.]
 [0. 0. 0. 0. 0. 6. 0. 0.]
 [0. 0. 0. 0. 0. 0. 7. 0.]
 [0. 0. 0. 0. 0. 0. 0. 8.]]
0.1
2


Create 6 more variables

In [39]:
two_scale = 2.*scale
four_scale = 4.*scale
eight_scale = 8.*scale
ten_scale = 10.*scale
fourteen_scale = 14.*scale
sixteen_scale = 16.*scale

X3_input = np.linspace(mu-two_scale, mu+two_scale, 1000)
X4_input = np.linspace(mu-four_scale, mu+four_scale, 1000)
X5_input = np.linspace(mu-eight_scale, mu+eight_scale, 1000)
X6_input = np.linspace(mu-ten_scale, mu+ten_scale, 1000)
X7_input = np.linspace(mu-fourteen_scale, mu+fourteen_scale, 1000)
X8_input = np.linspace(mu-sixteen_scale, mu+sixteen_scale, 1000)

In [40]:
X_8D = np.column_stack((X_input, X2_input, X3_input, X4_input, X5_input, X6_input, X7_input, X8_input))
X_8D.shape

(1000, 8)

Run prob

In [41]:
mcn.prob(X_8D)

array([5.74154747e-17, 5.93025416e-17, 6.12547826e-17, 6.32745559e-17,
       6.53643091e-17, 6.75265827e-17, 6.97640131e-17, 7.20793373e-17,
       7.44753961e-17, 7.69551388e-17, 7.95216271e-17, 8.21780397e-17,
       8.49276773e-17, 8.77739668e-17, 9.07204668e-17, 9.37708730e-17,
       9.69290232e-17, 1.00198903e-16, 1.03584654e-16, 1.07090574e-16,
       1.10721132e-16, 1.14480967e-16, 1.18374901e-16, 1.22407941e-16,
       1.26585293e-16, 1.30912363e-16, 1.35394769e-16, 1.40038352e-16,
       1.44849180e-16, 1.49833559e-16, 1.54998044e-16, 1.60349449e-16,
       1.65894858e-16, 1.71641632e-16, 1.77597428e-16, 1.83770204e-16,
       1.90168235e-16, 1.96800125e-16, 2.03674822e-16, 2.10801632e-16,
       2.18190231e-16, 2.25850685e-16, 2.33793463e-16, 2.42029454e-16,
       2.50569987e-16, 2.59426847e-16, 2.68612294e-16, 2.78139086e-16,
       2.88020496e-16, 2.98270337e-16, 3.08902983e-16, 3.19933392e-16,
       3.31377134e-16, 3.43250415e-16, 3.55570100e-16, 3.68353750e-16,
      

In [42]:
mcn.prob(X_8D).shape

(1000,)

***Multivariate Coupled Normal multiple distribution when d=2 and batch_size=3***

In [43]:
cn = MultivariateCoupledNormal(loc=[[0., 1.], [1., 2.], [2., 3.]], \
                               scale=[[0., 1.], [2., 3.], [4., 5.]],
                               kappa=0.1
                               )
cn

AssertionError: scale must be positive definite, but not necessarily symmetric.

In [ ]:
print(cn.dim)
print(cn.loc)
print(cn.scale)
print(cn.kappa)
print(cn.alpha)

In [ ]:
cn = CoupledNormal(loc=[[0., 1., 0., 1.], [1., 2., 1., 2.], [2., 3., 2., 3.]], \
                   scale=[[0., 1., 0., 1.], [2., 3., 2., 3.], [4., 5., 4., 5.]]
                   )
cn.dim

In [ ]:
print(cn.dim)
print(cn._normalized_term())

In [ ]:
# print(cn.prob(X_input))